In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

### Preparación de datasets

In [2]:
from sklearn.preprocessing import LabelEncoder

def dataset_prep(window=3, neutral=True):
    
    train = pd.read_csv('data/trainset.csv', index_col=0)
    test  = pd.read_csv('data/testset.csv', index_col=0)
    
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    
    for col in cols1:
        train[col+'w'] = train[col].rolling(window=window).sum()
        test[col+'w'] = test[col].rolling(window=window).sum()
    
    train['exc_ret'] = train['exc_ret'].shift(periods=-1)
    test['exc_ret']  = test['exc_ret'].shift(periods=-1)
    
    train.dropna(axis=0, inplace=True)
    test.dropna(axis=0, inplace=True)
    
    cols2 = [x+'w' for x in cols1]
    
    X = train[cols2]
    y = train.exc_ret
    Xt = test[cols2]
    yt = test.exc_ret
    
    le = LabelEncoder()
    y  = le.fit_transform(y)
    yt = le.fit_transform(yt)
    
    return X, Xt, y, yt    

### Entrenamiento de Clasificadores

In [3]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_roc

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier 

In [14]:
def train_classifier(X, Xt, y, yt):
    
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  LogisticRegression(),
                  CatBoostClassifier()]
    params1 = {  
        "n_estimators": st.randint(10,500),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 50),     # Maximum tree depth for base learners.
    }
    
    params2 = {  
        "n_estimators": st.randint(10,500),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 50),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    params4 = {}
    
    model_params = [params1, params2, params2, params3, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    for name, mod, params in zip(model_name, model_init, model_params):
        model = mod
        kf    = KFold(n_splits=2, shuffle=True, random_state=RS)
        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=kf,
                                   scoring='roc_auc', n_iter=2, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results.sort_values(by='Test Score', ascending=False)

In [18]:
import itertools
windows = np.arange(1, 3)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = dataset_prep(window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.7s finished


Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   20.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   20.2s finished


0:	learn: 0.6920860	total: 6.66ms	remaining: 6.66s
1:	learn: 0.6912919	total: 13.2ms	remaining: 6.59s
2:	learn: 0.6905420	total: 19.5ms	remaining: 6.49s
3:	learn: 0.6898710	total: 26ms	remaining: 6.48s
4:	learn: 0.6888259	total: 31.8ms	remaining: 6.33s
5:	learn: 0.6875129	total: 37.6ms	remaining: 6.23s
6:	learn: 0.6869348	total: 43.4ms	remaining: 6.16s
7:	learn: 0.6863831	total: 49.3ms	remaining: 6.12s
8:	learn: 0.6855110	total: 55ms	remaining: 6.06s
9:	learn: 0.6851182	total: 60.8ms	remaining: 6.02s
10:	learn: 0.6842678	total: 66.9ms	remaining: 6.01s
11:	learn: 0.6833723	total: 77.4ms	remaining: 6.37s
12:	learn: 0.6829225	total: 82.4ms	remaining: 6.26s
13:	learn: 0.6823976	total: 90.8ms	remaining: 6.39s
14:	learn: 0.6814359	total: 96.8ms	remaining: 6.35s
15:	learn: 0.6803952	total: 103ms	remaining: 6.32s
16:	learn: 0.6797356	total: 109ms	remaining: 6.28s
17:	learn: 0.6792605	total: 114ms	remaining: 6.24s
18:	learn: 0.6789783	total: 119ms	remaining: 6.13s
19:	learn: 0.6780960	total: 12

...

181:	learn: 0.6134654	total: 1.61s	remaining: 7.23s
182:	learn: 0.6132578	total: 1.63s	remaining: 7.25s
183:	learn: 0.6128226	total: 1.64s	remaining: 7.25s
184:	learn: 0.6126656	total: 1.64s	remaining: 7.23s
185:	learn: 0.6123203	total: 1.66s	remaining: 7.25s
186:	learn: 0.6119000	total: 1.67s	remaining: 7.26s
187:	learn: 0.6117464	total: 1.68s	remaining: 7.25s
188:	learn: 0.6114638	total: 1.69s	remaining: 7.26s
189:	learn: 0.6113186	total: 1.7s	remaining: 7.24s
190:	learn: 0.6110902	total: 1.71s	remaining: 7.22s
191:	learn: 0.6110148	total: 1.71s	remaining: 7.21s
192:	learn: 0.6109693	total: 1.72s	remaining: 7.2s
193:	learn: 0.6107701	total: 1.73s	remaining: 7.2s
194:	learn: 0.6105007	total: 1.74s	remaining: 7.2s
195:	learn: 0.6102724	total: 1.75s	remaining: 7.2s
196:	learn: 0.6102145	total: 1.76s	remaining: 7.19s
197:	learn: 0.6101426	total: 1.77s	remaining: 7.18s
198:	learn: 0.6100364	total: 1.78s	remaining: 7.17s
199:	learn: 0.6099597	total: 1.79s	remaining: 7.17s
200:	learn: 0.609

349:	learn: 0.5881374	total: 3.43s	remaining: 6.36s
350:	learn: 0.5880076	total: 3.43s	remaining: 6.34s
351:	learn: 0.5878798	total: 3.44s	remaining: 6.33s
352:	learn: 0.5877003	total: 3.44s	remaining: 6.31s
353:	learn: 0.5875409	total: 3.45s	remaining: 6.29s
354:	learn: 0.5875354	total: 3.46s	remaining: 6.28s
355:	learn: 0.5873646	total: 3.46s	remaining: 6.26s
356:	learn: 0.5872792	total: 3.47s	remaining: 6.24s
357:	learn: 0.5871145	total: 3.47s	remaining: 6.23s
358:	learn: 0.5869407	total: 3.48s	remaining: 6.21s
359:	learn: 0.5868709	total: 3.49s	remaining: 6.2s
360:	learn: 0.5868102	total: 3.49s	remaining: 6.18s
361:	learn: 0.5867023	total: 3.5s	remaining: 6.17s
362:	learn: 0.5866268	total: 3.51s	remaining: 6.15s
363:	learn: 0.5865977	total: 3.51s	remaining: 6.14s
364:	learn: 0.5864408	total: 3.52s	remaining: 6.12s
365:	learn: 0.5863830	total: 3.52s	remaining: 6.11s
366:	learn: 0.5862939	total: 3.53s	remaining: 6.09s
367:	learn: 0.5860570	total: 3.54s	remaining: 6.08s
368:	learn: 0.

523:	learn: 0.5695975	total: 4.61s	remaining: 4.19s
524:	learn: 0.5695690	total: 4.61s	remaining: 4.17s
525:	learn: 0.5695219	total: 4.62s	remaining: 4.16s
526:	learn: 0.5694423	total: 4.63s	remaining: 4.15s
527:	learn: 0.5694051	total: 4.63s	remaining: 4.14s
528:	learn: 0.5692079	total: 4.64s	remaining: 4.13s
529:	learn: 0.5691604	total: 4.64s	remaining: 4.12s
530:	learn: 0.5687653	total: 4.65s	remaining: 4.1s
531:	learn: 0.5687394	total: 4.65s	remaining: 4.09s
532:	learn: 0.5687087	total: 4.66s	remaining: 4.08s
533:	learn: 0.5686729	total: 4.67s	remaining: 4.07s
534:	learn: 0.5686282	total: 4.68s	remaining: 4.07s
535:	learn: 0.5686175	total: 4.68s	remaining: 4.05s
536:	learn: 0.5685574	total: 4.7s	remaining: 4.05s
537:	learn: 0.5685351	total: 4.7s	remaining: 4.04s
538:	learn: 0.5683408	total: 4.71s	remaining: 4.03s
539:	learn: 0.5683176	total: 4.71s	remaining: 4.01s
540:	learn: 0.5679385	total: 4.72s	remaining: 4s
541:	learn: 0.5677405	total: 4.73s	remaining: 3.99s
542:	learn: 0.5676

685:	learn: 0.5544913	total: 5.59s	remaining: 2.56s
686:	learn: 0.5544235	total: 5.59s	remaining: 2.55s
687:	learn: 0.5543639	total: 5.6s	remaining: 2.54s
688:	learn: 0.5543252	total: 5.61s	remaining: 2.53s
689:	learn: 0.5541311	total: 5.62s	remaining: 2.52s
690:	learn: 0.5539881	total: 5.62s	remaining: 2.51s
691:	learn: 0.5537162	total: 5.63s	remaining: 2.5s
692:	learn: 0.5536603	total: 5.63s	remaining: 2.5s
693:	learn: 0.5534130	total: 5.64s	remaining: 2.49s
694:	learn: 0.5533794	total: 5.64s	remaining: 2.48s
695:	learn: 0.5533663	total: 5.65s	remaining: 2.47s
696:	learn: 0.5533571	total: 5.66s	remaining: 2.46s
697:	learn: 0.5533041	total: 5.66s	remaining: 2.45s
698:	learn: 0.5532193	total: 5.67s	remaining: 2.44s
699:	learn: 0.5531744	total: 5.67s	remaining: 2.43s
700:	learn: 0.5531630	total: 5.68s	remaining: 2.42s
701:	learn: 0.5530285	total: 5.68s	remaining: 2.41s
702:	learn: 0.5527899	total: 5.69s	remaining: 2.4s
703:	learn: 0.5527271	total: 5.7s	remaining: 2.39s
704:	learn: 0.552

855:	learn: 0.5399648	total: 6.55s	remaining: 1.1s
856:	learn: 0.5398825	total: 6.55s	remaining: 1.09s
857:	learn: 0.5397823	total: 6.56s	remaining: 1.08s
858:	learn: 0.5397487	total: 6.57s	remaining: 1.08s
859:	learn: 0.5397013	total: 6.57s	remaining: 1.07s
860:	learn: 0.5396372	total: 6.58s	remaining: 1.06s
861:	learn: 0.5395997	total: 6.58s	remaining: 1.05s
862:	learn: 0.5395954	total: 6.59s	remaining: 1.05s
863:	learn: 0.5393778	total: 6.6s	remaining: 1.04s
864:	learn: 0.5392899	total: 6.6s	remaining: 1.03s
865:	learn: 0.5392479	total: 6.61s	remaining: 1.02s
866:	learn: 0.5391345	total: 6.61s	remaining: 1.01s
867:	learn: 0.5389493	total: 6.62s	remaining: 1.01s
868:	learn: 0.5389393	total: 6.62s	remaining: 999ms
869:	learn: 0.5389295	total: 6.63s	remaining: 991ms
870:	learn: 0.5385275	total: 6.63s	remaining: 983ms
871:	learn: 0.5385272	total: 6.64s	remaining: 975ms
872:	learn: 0.5384086	total: 6.65s	remaining: 967ms
873:	learn: 0.5382125	total: 6.65s	remaining: 959ms
874:	learn: 0.5

In [19]:
resultado = resultado.sort_values(by='Test Score', ascending=False)
best = resultado.Estimator[0]
resultado = resultado.iloc[:, :-1]

In [20]:
resultado

,Model,Window,Neutral,Train Score,Test Score,Params
0,Random Forest,1,0,0.474600,0.554144,"{'max_depth': 17, 'n_estimators': 123}"
2,LightGBM,1,0,0.435453,0.501905,"{'colsample_bytree': 0.8745784537080196, 'gamma': 7.612611654813181, 'learning_rate': 0.1346907279327682, 'max_depth': 20, 'min_child_weight': 12.414459909745108, 'n_estimators': 336, 'reg_alpha': 0.6128685044735249, 'subsample': 0.834840352558528}"
1,XGBoost,1,1,0.500000,0.500000,"{'colsample_bytree': 0.9980524260311265, 'gamma': 8.511316237114674, 'learning_rate': 0.30558324756818117, 'max_depth': 14, 'min_child_weight': 9.493132446692462, 'n_estimators': 296, 'reg_alpha': 9.20233742986444, 'subsample': 0.946444576622583}"
1,XGBoost,1,0,0.500000,0.500000,"{'colsample_bytree': 0.8898035897834322, 'gamma': 4.755799363082529, 'learning_rate': 0.10161881601577487, 'max_depth': 22, 'min_child_weight': 14.25352490873422, 'n_estimators': 300, 'reg_alpha': 9.339346301589629, 'subsample': 0.7424435431713327}"
2,LightGBM,2,0,0.500000,0.500000,"{'colsample_bytree': 0.6723909498086236, 'gamma': 9.34479360093693, 'learning_rate': 0.30172366862573036, 'max_depth': 30, 'min_child_weight': 19.27145334850182, 'n_estimators': 389, 'reg_alpha': 8.695943237262558, 'subsample': 0.8736298679200143}"
1,XGBoost,2,0,0.500000,0.500000,"{'colsample_bytree': 0.91458683013468, 'gamma': 2.1272312240167333, 'learning_rate': 0.21341100528571622, 'max_depth': 32, 'min_child_weight': 6.180935582813735, 'n_estimators': 120, 'reg_alpha': 5.6232636726415555, 'subsample': 0.9778349356101632}"
2,LightGBM,1,1,0.500000,0.500000,"{'colsample_bytree': 0.716271597672722, 'gamma': 2.4874443158414996, 'learning_rate': 0.10997616310546719, 'max_depth': 27, 'min_child_weight': 20.2104460250546, 'n_estimators': 170, 'reg_alpha': 8.329946906546331, 'subsample': 0.9426391267305977}"
1,XGBoost,2,1,0.500000,0.500000,"{'colsample_bytree': 0.9816156206897297, 'gamma': 4.2519223740790855, 'learning_rate': 0.25815330857786006, 'max_depth': 19, 'min_child_weight': 6.3451732936369485, 'n_estimators': 172, 'reg_alpha': 9.256437667558124, 'subsample': 0.9810968008855452}"
4,Catboost,1,0,0.454237,0.486504,{}
4,Catboost,2,0,0.464920,0.482641,{}
